In [4]:
import ironforest as ss
import plotly.express as px
from dubious.distributions import Normal, LogNormal, Uniform, Beta 
from dubious.core import Uncertain, Context
rng = ss.random.Generator.from_seed(0)


In [5]:
ctx = Context()

length = Uncertain(Normal(10, 1), ctx=ctx)
width  = Uncertain(LogNormal(mu=1.6, sigma=0.2), ctx=ctx)

length.corr(width, 0.9)

area = length * width

print("Global:", area.sensitivity(20_000))
print("Length ~12:", area.local_sensitivity({length: 12.0}, n=20_000, k=100))
print("Width ~3:", area.local_sensitivity({width: 3.0}, n=20_000, k=100))
print("Both:", area.local_sensitivity({length: 12.0, width.node_id: 7.0}, n=20_000, k=100))

Global: {1: 0.9378627199365266, 2: 0.9881813835825601}


AttributeError: module 'ironforest' has no attribute 'column_stack'

In [ ]:
from dubious.distributions import Normal
from dubious.core import Uncertain, Context

ctx = Context()

x = Uncertain(Normal(0, 1), ctx=ctx)
y = Uncertain(Normal(10, 2), ctx=ctx)
z = x * y + 3

ctx.freeze(20_000)
print("run1:", z.mean(), z.quantile(0.05))
print("run2:", z.mean(), z.quantile(0.05))  # same

print(x.frozen)

ctx.unfreeze()
print("run3:", z.mean(), z.quantile(0.05))  # likely slightly different


In [ ]:
from dubious.distributions import Normal
from dubious.core import Uncertain, Context
from dubious.core import Sampler

ctx = Context()

x = Uncertain(Normal(0, 1), ctx=ctx)
y = Uncertain(Normal(10, 2), ctx=ctx)
z = x * y

sampler = Sampler()

for i in range(0,6):
    print(z.draw())

ctx.freeze(5)
for i in range(0,6):
    print(z.draw())
    z.ctx.redraw()

ctx.unfreeze()
for i in range(0,6):
    print(z.draw())


In [ ]:
data = area.sample(200000)
print(data.median())
fig = px.histogram(
    data,
    nbins=200,
)

fig.show()

In [ ]:
#two non-gassian marginals correlated using copula
from dubious.distributions import Beta, LogNormal
from dubious.core import Uncertain, Context

ctx = Context()

conv = Uncertain(Beta(20,80), ctx=ctx)
traffic = Uncertain(LogNormal(8.0,0.4), ctx=ctx)

conv.corr(traffic, 0.7)

sales = conv * traffic

print(f"Mean: {sales.mean()}")
print(f"p10, p90: {sales.quantile(0.1)}, {sales.quantile(0.9)}")

data = sales.sample(10_000)
fig = px.histogram(
    data,
    nbins=200,
)

fig.show()

In [ ]:
from dubious.distributions import Normal, Beta
from dubious.core import Uncertain, Context

ctx = Context()

#We can define distribution parameters with other distributions.
normal = Normal(10, 1)
beta = Beta(normal,1)

x = Uncertain(normal, ctx=ctx)
y = Uncertain(beta, ctx=ctx)

#Apply some arithmetic.
x = x*y

#We can also use uncertain distributions to define parameters.
normal3 = Normal(2, 3)
print(normal3.mean())


In [ ]:
from dubious.distributions import Normal, Beta
from dubious.core import Uncertain, Context

ctx = Context()
ctx.freeze(5)

normal = Normal(10, 1)
x = Uncertain(normal, ctx=ctx)
# x.sample(5)
for i in range(0,7):
    print(x.redraw())

print(f"float: {float(x)}")